In [1]:

import numpy as np
import gurobipy as gp
from gurobipy import GRB

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from problem import Problem
from data import DataGen

from parameterized import ParameterizedExplainer

from models import * 

%load_ext autoreload
%autoreload 2


np.random.seed(0)
torch.manual_seed(0)

import warnings
warnings.filterwarnings("ignore")

In [2]:
n_nodes = 5
n_features = 5

h = 1
b = 5
H = torch.tensor([h for i in range(n_nodes)])
B = torch.tensor([b for i in range(n_nodes)])  

problem_ = Problem(H, B, n_nodes)

n_data = 1000
n_test = 1000
data_generator = DataGen(n_features, n_nodes)
X_train, Y_train, X_test, Y_test = data_generator.get_test_train(n_data, n_test)

# base forecast
# print("training mse")
# two_stage_forecast = train_two_stage(problem_, X_train, Y_train, X_test, Y_test, EPOCHS=300)

print() 
print("training paraeterized model")
model = ParameterizedExplainer(data_generator.data_model, X_train, Y_train, problem_)
model.train(EPOCHS = 3000, lr=1e-3)

print("parameterization   :", model.mean_forecast.detach().numpy())
print("base forecast param:", model.mean_forecast_base.detach().numpy())
print("quantiles:          ", nn.functional.sig moid(model.quantiles).detach().numpy())


training paraeterized model
epoch 0 err: 2.1043792
epoch 10 err: 2.134461
epoch 20 err: 2.1463327
epoch 30 err: 2.1332
epoch 40 err: 2.1145508
epoch 50 err: 2.1301708
epoch 60 err: 2.0447626
epoch 70 err: 2.0487046
epoch 80 err: 2.0428536
epoch 90 err: 2.0261335
epoch 100 err: 2.0112727
epoch 110 err: 1.9959346
epoch 120 err: 1.952572
epoch 130 err: 1.9790673
epoch 140 err: 1.9889928
epoch 150 err: 1.9425492
epoch 160 err: 1.9205323
epoch 170 err: 1.8732754
epoch 180 err: 1.864607
epoch 190 err: 1.9209106
epoch 200 err: 1.8327763
epoch 210 err: 1.8559242
epoch 220 err: 1.8367813
epoch 230 err: 1.8176728
epoch 240 err: 1.7946948
epoch 250 err: 1.7650588
epoch 260 err: 1.7791965
epoch 270 err: 1.7170824
epoch 280 err: 1.7504016
epoch 290 err: 1.7103752
epoch 300 err: 1.6712368
epoch 310 err: 1.683952
epoch 320 err: 1.6622375
epoch 330 err: 1.6500496
epoch 340 err: 1.6452761
epoch 350 err: 1.6289674
epoch 360 err: 1.6403977
epoch 370 err: 1.5967662
epoch 380 err: 1.5848922
epoch 390 err:

In [4]:
model.train(EPOCHS = 1, lr=1e-4)
print("parameterization   :", model.mean_forecast.detach().numpy())
print("base forecast param:", model.mean_forecast_base.detach().numpy())
print("quantiles:          ", nn.functional.sigmoid(model.quantiles).detach().numpy())

epoch 0 err: 0.5437277
parameterization   : [[ 0.01776451 -0.04704283  0.34938976  0.6404606   0.30778393]
 [-0.2350912   0.24869506  0.47298408  0.02390896  0.17002119]
 [-0.10253526 -0.12594922  0.04081867  0.4595195   0.3503231 ]
 [ 0.08862431  0.03031999  0.17056733  0.4861437   0.39314583]
 [-0.09603944 -0.0481602  -0.06095366  0.3935478  -0.04413341]]
base forecast param: [[ 0.23629919  0.07446777  0.47669196  0.5370874  -0.05622284]
 [ 0.60023504  0.61320883 -0.13529459 -0.13269742  0.35343564]
 [ 0.21387649  0.03044944  0.5327656   0.63727444  0.5550722 ]
 [-0.0529624   0.07460525  0.4968069   0.06173573 -0.21327318]
 [ 0.01544752  0.27547976  0.4898046   0.70485145  0.37523177]]
quantiles:           [0.6104788  0.66611    0.62551093 0.7037672  0.50585735]


In [5]:
means = model.base_forecast(X_train)
c = 5
means = ((model.mean_forecast * c + model.mean_forecast_base) @ means.T).T
print(means)

tensor([[-0.1409,  1.6270, -0.5252,  0.1982,  0.0304],
        [ 0.6555,  1.1419,  0.1236,  0.6816,  0.4685],
        [ 2.4124,  1.3884,  0.5301,  1.7644, -0.3429],
        ...,
        [-0.1586,  1.8318, -0.5913,  0.2232,  0.0342],
        [ 3.1944,  0.7309,  1.7299,  2.4947,  1.7963],
        [ 0.0303,  1.3229, -0.1810,  0.3077,  0.0368]],
       grad_fn=<PermuteBackward0>)


In [6]:
Y_train

tensor([[0.3255, 0.0000, 0.0628, 0.0800, 0.0388],
        [0.7347, 0.0459, 0.1164, 0.0918, 0.0699],
        [0.0626, 0.1083, 0.4328, 0.8582, 0.1906],
        ...,
        [0.0354, 0.0858, 0.0584, 0.0000, 1.4566],
        [0.7442, 0.1121, 0.0000, 0.0138, 0.0877],
        [0.0000, 0.1640, 0.0000, 0.0000, 0.9705]], grad_fn=<ReluBackward0>)

In [7]:
print("base forecast param:", model.mean_forecast.detach().numpy())
print("parameterization:   ", model.mean_forecast_base.detach().numpy())
print("quantiles:          ", nn.functional.sigmoid(model.quantiles).detach().numpy())

base forecast param: [[ 0.01776451 -0.04704283  0.34938976  0.6404606   0.30778393]
 [-0.2350912   0.24869506  0.47298408  0.02390896  0.17002119]
 [-0.10253526 -0.12594922  0.04081867  0.4595195   0.3503231 ]
 [ 0.08862431  0.03031999  0.17056733  0.4861437   0.39314583]
 [-0.09603944 -0.0481602  -0.06095366  0.3935478  -0.04413341]]
parameterization:    [[ 0.23629919  0.07446777  0.47669196  0.5370874  -0.05622284]
 [ 0.60023504  0.61320883 -0.13529459 -0.13269742  0.35343564]
 [ 0.21387649  0.03044944  0.5327656   0.63727444  0.5550722 ]
 [-0.0529624   0.07460525  0.4968069   0.06173573 -0.21327318]
 [ 0.01544752  0.27547976  0.4898046   0.70485145  0.37523177]]
quantiles:           [0.6104788  0.66611    0.62551093 0.7037672  0.50585735]


In [8]:
# c = np.random.rand() * self.problem.B[0]
for c in np.arange(0, b, 0.1):
# for c in [0]:
    cross_cost = torch.tensor([[0,c],[c,0]])
    cross_costs_ordered = problem_.get_cross_costs_ordered(cross_cost)

    f = model.predict(X_train, c)

    error = problem_.fulfilment_loss(f, Y_train, cross_cost, cross_costs_ordered) / (len(Y_train))

    print("c:", c, "err:", error.item())

IndexError: index 2 is out of bounds for dimension 1 with size 2

In [ ]:
means = model.base_forecast(X_train)
means

tensor([[0.1710, 0.0000],
        [0.1720, 0.0000],
        [0.1781, 0.0000],
        ...,
        [0.1749, 0.0000],
        [0.1683, 0.0000],
        [0.1825, 0.0000]], grad_fn=<ReluBackward0>)

In [ ]:
Y_train

tensor([[0.3470, 0.7594],
        [0.2696, 0.8594],
        [0.3661, 0.0000],
        ...,
        [0.1951, 0.3896],
        [0.3075, 0.9694],
        [0.0705, 0.0000]], dtype=torch.float64, grad_fn=<ReluBackward0>)

In [ ]:
print("base forecast param:", model.mean_forecast.detach().numpy())
print("parameterization:   ", model.mean_forecast_base.detach().numpy())
print("quantiles:          ", nn.functional.sigmoid(model.quantiles).detach().numpy())

base forecast param: [[0.64092517 0.2909161 ]
 [0.6801625  0.3556071 ]]
parameterization:    [[1.0300598 0.9904706]
 [1.1528603 0.5874913]]
quantiles:           [0.60936236 0.65545124]
